In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('/cdthome/iuo722/accel')
from pathlib import Path
from accel.read_data import prep_data
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
from sklearn.model_selection import ParameterGrid

from pytorch_tabnet.tab_model import TabNetRegressor

np.random.seed(42)

In [2]:
def score(y_true,y_pred): 
    """ Function to print the metrics of interest of the model """
    mse = mean_squared_error(y_true, y_pred) #set score here and not below if using MSE in GridCV
    r2 = r2_score(y_true, y_pred)
    ev = explained_variance_score(y_true, y_pred)
    print("MSE is: ", mse)
    print("R2 is: ", r2)
    print("Explained variance is:", ev)

### Load the data

In [3]:
PATH = '/cdtshared/wearables/students/group5/'

X_train, X_val, X_test, y_train, y_val, y_test, mean_mode = prep_data(PATH + "eliminated-missing-participants.csv",normalise=True, one_hot=True)

<class 'pandas.core.frame.DataFrame'>


### Model tuning

In [4]:
n_d = [8, 16, 32, 64]
#n_a = [8,16,32, 64]
n_steps = [3,5, 8]
gamma = [1.0,1.3,1.7,2.0]

# Create the grid
param_grid = {'n_d': n_d,
               'n_steps': n_steps,
               'gamma': gamma}

In [5]:
"""
# Attempt to use the scikit-learn built in functions
model = TabNetRegressor()
model.fit(X_train.to_numpy(), y_train.values.reshape(-1,1))

clf = GridSearchCV(model, param_grid)
clf.fit(X_train.to_numpy(), y_train.values.reshape(-1,1),scoring=mean_squared_error)

# Try with Bayesian optimisation for faster computation of tuning
opt = BayesSearchCV(model, param_grid, n_iter=30, cv=5, verbose=1)
opt.fit(X_train, y_train)
"""

'\n# Attempt to use the scikit-learn built in functions\nmodel = TabNetRegressor()\nmodel.fit(X_train.to_numpy(), y_train.values.reshape(-1,1))\n\nclf = GridSearchCV(model, param_grid)\nclf.fit(X_train.to_numpy(), y_train.values.reshape(-1,1),scoring=mean_squared_error)\n\n# Try with Bayesian optimisation for faster computation of tuning\nopt = BayesSearchCV(model, param_grid, n_iter=30, cv=5, verbose=1)\nopt.fit(X_train, y_train)\n'

In [ ]:
grid = ParameterGrid(param_grid)

search_results = pd.DataFrame() 
for params in grid:
    params['n_a'] = params['n_d'] # n_a=n_d always per the paper
    tabnet = TabNetRegressor()
    tabnet.set_params(**params)
    tabnet.fit(X_train.to_numpy(), y_train.values.reshape(-1,1), max_epochs=60)
    score = mean_squared_error(y_val.to_numpy(), tabnet.predict(X_val.to_numpy()))
    
    results = pd.DataFrame([params])
    results['score'] = score
    search_results = search_results.append(results)
    search_results.to_csv("results-larger-grid.csv")
    

Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 312.37536|  0:00:03s
epoch 1  | loss: 81.69316|  0:00:06s
epoch 2  | loss: 77.45436|  0:00:09s
epoch 3  | loss: 76.54649|  0:00:12s
epoch 4  | loss: 75.49936|  0:00:15s
epoch 5  | loss: 74.45534|  0:00:19s
epoch 6  | loss: 73.7138 |  0:00:22s
epoch 7  | loss: 73.11408|  0:00:25s
epoch 8  | loss: 72.70931|  0:00:28s
epoch 9  | loss: 72.29734|  0:00:31s
epoch 10 | loss: 71.88532|  0:00:34s
epoch 11 | loss: 71.69349|  0:00:38s
epoch 12 | loss: 71.25032|  0:00:41s
epoch 13 | loss: 71.16936|  0:00:44s
epoch 14 | loss: 71.21986|  0:00:47s
epoch 15 | loss: 70.78746|  0:00:50s
epoch 16 | loss: 70.63534|  0:00:53s
epoch 17 | loss: 70.61977|  0:00:56s
epoch 18 | loss: 70.42246|  0:00:59s
epoch 19 | loss: 70.043  |  0:01:03s
epoch 20 | loss: 70.09652|  0:01:06s
epoch 21 | loss: 69.54108|  0:01:09s
epoch 22 | loss: 69.22034|  0:01:12s
epoch 23 | loss: 69.01707|  0:01:15s
epoch 24 | loss: 68

epoch 14 | loss: 74.86076|  0:01:20s
epoch 15 | loss: 75.77875|  0:01:26s
epoch 16 | loss: 75.09118|  0:01:32s
epoch 17 | loss: 75.02481|  0:01:37s
epoch 18 | loss: 74.98275|  0:01:43s
epoch 19 | loss: 75.11447|  0:01:49s
epoch 20 | loss: 74.99015|  0:01:54s
epoch 21 | loss: 74.64398|  0:02:00s
epoch 22 | loss: 74.64298|  0:02:05s
epoch 23 | loss: 74.6953 |  0:02:10s
epoch 24 | loss: 74.53597|  0:02:16s
epoch 25 | loss: 74.85296|  0:02:21s
epoch 26 | loss: 74.48037|  0:02:26s
epoch 27 | loss: 74.44211|  0:02:32s
epoch 28 | loss: 74.95167|  0:02:37s
epoch 29 | loss: 74.31039|  0:02:42s
epoch 30 | loss: 74.26841|  0:02:48s
epoch 31 | loss: 74.32581|  0:02:53s
epoch 32 | loss: 74.62068|  0:02:58s
epoch 33 | loss: 74.57591|  0:03:03s
epoch 34 | loss: 74.24712|  0:03:09s
epoch 35 | loss: 74.18614|  0:03:14s
epoch 36 | loss: 74.23686|  0:03:19s
epoch 37 | loss: 74.39203|  0:03:25s
epoch 38 | loss: 74.57047|  0:03:30s
epoch 39 | loss: 74.21797|  0:03:36s
epoch 40 | loss: 74.73837|  0:03:41s
e

epoch 31 | loss: 61.71572|  0:02:05s
epoch 32 | loss: 61.2905 |  0:02:10s
epoch 33 | loss: 60.28201|  0:02:15s
epoch 34 | loss: 59.85488|  0:02:19s
epoch 35 | loss: 59.49552|  0:02:24s
epoch 36 | loss: 58.63632|  0:02:29s
epoch 37 | loss: 58.06444|  0:02:34s
epoch 38 | loss: 57.66857|  0:02:38s
epoch 39 | loss: 57.21827|  0:02:42s
epoch 40 | loss: 57.35132|  0:02:46s
epoch 41 | loss: 56.67903|  0:02:50s
epoch 42 | loss: 55.5095 |  0:02:54s
epoch 43 | loss: 55.44364|  0:02:57s
epoch 44 | loss: 55.19598|  0:03:01s
epoch 45 | loss: 54.50453|  0:03:05s
epoch 46 | loss: 54.52905|  0:03:08s
epoch 47 | loss: 53.73705|  0:03:12s
epoch 48 | loss: 53.2034 |  0:03:16s
epoch 49 | loss: 52.82004|  0:03:19s
epoch 50 | loss: 52.46177|  0:03:23s
epoch 51 | loss: 52.06117|  0:03:27s
epoch 52 | loss: 52.25378|  0:03:30s
epoch 53 | loss: 51.13118|  0:03:34s
epoch 54 | loss: 51.26238|  0:03:37s
epoch 55 | loss: 50.80119|  0:03:41s
epoch 56 | loss: 50.65709|  0:03:45s
epoch 57 | loss: 50.85032|  0:03:49s
e

epoch 48 | loss: 38.54979|  0:02:20s
epoch 49 | loss: 38.34441|  0:02:24s
epoch 50 | loss: 37.51291|  0:02:27s
epoch 51 | loss: 36.92427|  0:02:29s
epoch 52 | loss: 36.8115 |  0:02:32s
epoch 53 | loss: 36.52337|  0:02:34s
epoch 54 | loss: 35.52908|  0:02:37s
epoch 55 | loss: 35.33866|  0:02:40s
epoch 56 | loss: 34.38518|  0:02:42s
epoch 57 | loss: 34.42707|  0:02:45s
epoch 58 | loss: 33.94797|  0:02:47s
epoch 59 | loss: 33.58185|  0:02:50s
epoch 60 | loss: 33.15498|  0:02:53s
epoch 61 | loss: 32.40138|  0:02:55s
epoch 62 | loss: 31.57974|  0:02:58s
epoch 63 | loss: 31.97055|  0:03:01s
epoch 64 | loss: 31.49951|  0:03:04s
epoch 65 | loss: 31.36065|  0:03:07s
epoch 66 | loss: 31.17927|  0:03:10s
epoch 67 | loss: 31.05164|  0:03:13s
epoch 68 | loss: 30.91767|  0:03:16s
epoch 69 | loss: 30.88273|  0:03:19s
epoch 70 | loss: 30.09005|  0:03:22s
epoch 71 | loss: 29.79993|  0:03:24s
epoch 72 | loss: 29.90763|  0:03:27s
epoch 73 | loss: 29.94107|  0:03:30s
epoch 74 | loss: 28.86526|  0:03:33s
e

epoch 65 | loss: 51.61065|  0:05:52s
epoch 66 | loss: 51.43685|  0:05:57s
epoch 67 | loss: 51.04534|  0:06:02s
epoch 68 | loss: 51.12594|  0:06:07s
epoch 69 | loss: 50.52495|  0:06:12s
epoch 70 | loss: 49.80589|  0:06:17s
epoch 71 | loss: 49.36082|  0:06:22s
epoch 72 | loss: 48.18191|  0:06:27s
epoch 73 | loss: 48.35283|  0:06:32s
epoch 74 | loss: 48.10306|  0:06:37s
epoch 75 | loss: 47.24865|  0:06:42s
epoch 76 | loss: 46.65369|  0:06:47s
epoch 77 | loss: 46.00038|  0:06:51s
epoch 78 | loss: 45.45222|  0:06:56s
epoch 79 | loss: 44.95182|  0:07:01s
epoch 80 | loss: 44.26415|  0:07:06s
epoch 81 | loss: 43.38165|  0:07:11s
epoch 82 | loss: 42.61244|  0:07:16s
epoch 83 | loss: 42.27978|  0:07:21s
epoch 84 | loss: 41.81759|  0:07:26s
epoch 85 | loss: 42.1396 |  0:07:31s
epoch 86 | loss: 42.34114|  0:07:36s
epoch 87 | loss: 41.98118|  0:07:41s
epoch 88 | loss: 41.42957|  0:07:46s
epoch 89 | loss: 40.41868|  0:07:50s
epoch 90 | loss: 39.3108 |  0:07:55s
epoch 91 | loss: 39.21797|  0:08:00s
e

...

epoch 79 | loss: 57.22862|  0:05:35s
epoch 80 | loss: 57.5161 |  0:05:39s
epoch 81 | loss: 57.51254|  0:05:43s
epoch 82 | loss: 57.075  |  0:05:47s
epoch 83 | loss: 57.2467 |  0:05:51s
epoch 84 | loss: 56.5519 |  0:05:55s
epoch 85 | loss: 56.53337|  0:05:59s
epoch 86 | loss: 56.89923|  0:06:03s
epoch 87 | loss: 56.57452|  0:06:07s
epoch 88 | loss: 56.87436|  0:06:11s
epoch 89 | loss: 56.36863|  0:06:15s
epoch 90 | loss: 56.54715|  0:06:19s
epoch 91 | loss: 56.15603|  0:06:23s
epoch 92 | loss: 55.9424 |  0:06:27s
epoch 93 | loss: 56.07956|  0:06:31s
epoch 94 | loss: 56.02763|  0:06:35s
epoch 95 | loss: 56.31972|  0:06:39s
epoch 96 | loss: 56.55101|  0:06:43s
epoch 97 | loss: 56.57901|  0:06:47s
epoch 98 | loss: 56.89295|  0:06:51s
epoch 99 | loss: 56.72315|  0:06:55s
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 264.78264|  0:00:05s
epoch 1  | loss: 89.92706|  0:00:11s
epoch 2  | loss: 86.86117|  0:00:16s
epoch 3  | loss: 86

epoch 96 | loss: 46.5602 |  0:04:29s
epoch 97 | loss: 46.72772|  0:04:32s
epoch 98 | loss: 46.50009|  0:04:34s
epoch 99 | loss: 46.32722|  0:04:37s
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 315.38642|  0:00:03s
epoch 1  | loss: 86.79656|  0:00:07s
epoch 2  | loss: 84.93751|  0:00:11s
epoch 3  | loss: 83.81853|  0:00:15s
epoch 4  | loss: 82.66033|  0:00:19s
epoch 5  | loss: 80.48818|  0:00:22s
epoch 6  | loss: 78.77209|  0:00:26s
epoch 7  | loss: 76.74848|  0:00:29s
epoch 8  | loss: 75.36483|  0:00:33s
epoch 9  | loss: 74.40809|  0:00:37s
epoch 10 | loss: 73.43167|  0:00:41s
epoch 11 | loss: 72.88184|  0:00:45s
epoch 12 | loss: 72.4716 |  0:00:48s
epoch 13 | loss: 72.36311|  0:00:52s
epoch 14 | loss: 71.8846 |  0:00:56s
epoch 15 | loss: 71.66741|  0:01:00s
epoch 16 | loss: 71.42203|  0:01:04s
epoch 17 | loss: 71.29851|  0:01:07s
epoch 18 | loss: 71.17596|  0:01:11s
epoch 19 | loss: 70.94089|  0:01:15s
epoch 20 | loss: 70

epoch 10 | loss: 71.50597|  0:00:34s
epoch 11 | loss: 71.24383|  0:00:37s
epoch 12 | loss: 70.83292|  0:00:40s
epoch 13 | loss: 70.45778|  0:00:44s
epoch 14 | loss: 70.21313|  0:00:47s
epoch 15 | loss: 69.01892|  0:00:50s
epoch 16 | loss: 68.40432|  0:00:53s
epoch 17 | loss: 67.75156|  0:00:56s
epoch 18 | loss: 66.5172 |  0:00:59s
epoch 19 | loss: 65.96439|  0:01:02s
epoch 20 | loss: 64.54594|  0:01:06s
epoch 21 | loss: 63.20927|  0:01:09s
epoch 22 | loss: 62.11961|  0:01:12s
epoch 23 | loss: 61.03058|  0:01:15s
epoch 24 | loss: 59.58116|  0:01:19s
epoch 25 | loss: 58.37465|  0:01:22s
epoch 26 | loss: 57.04529|  0:01:25s
epoch 27 | loss: 56.09862|  0:01:28s
epoch 28 | loss: 55.00239|  0:01:31s
epoch 29 | loss: 54.38779|  0:01:34s
epoch 30 | loss: 53.42028|  0:01:37s
epoch 31 | loss: 51.4962 |  0:01:41s
epoch 32 | loss: 50.97379|  0:01:44s
epoch 33 | loss: 49.76829|  0:01:47s
epoch 34 | loss: 48.61156|  0:01:50s
epoch 35 | loss: 47.73953|  0:01:53s
epoch 36 | loss: 46.91133|  0:01:56s
e

epoch 27 | loss: 70.35786|  0:02:30s
epoch 28 | loss: 69.8565 |  0:02:34s
epoch 29 | loss: 69.83153|  0:02:39s
epoch 30 | loss: 69.77896|  0:02:44s
epoch 31 | loss: 68.69457|  0:02:49s
epoch 32 | loss: 68.22579|  0:02:54s
epoch 33 | loss: 67.51633|  0:02:59s
epoch 34 | loss: 67.50294|  0:03:04s
epoch 35 | loss: 66.9921 |  0:03:09s
epoch 36 | loss: 66.59732|  0:03:14s
epoch 37 | loss: 66.04363|  0:03:19s
epoch 38 | loss: 65.64858|  0:03:24s
epoch 39 | loss: 64.48944|  0:03:29s
epoch 40 | loss: 64.04979|  0:03:34s
epoch 41 | loss: 63.61254|  0:03:39s
epoch 42 | loss: 63.3991 |  0:03:44s
epoch 43 | loss: 62.16613|  0:03:49s
epoch 44 | loss: 61.64837|  0:03:54s
epoch 45 | loss: 60.68765|  0:03:59s
epoch 46 | loss: 59.68111|  0:04:04s
epoch 47 | loss: 58.8654 |  0:04:09s
epoch 48 | loss: 58.55645|  0:04:14s
epoch 49 | loss: 58.82473|  0:04:19s
epoch 50 | loss: 59.08546|  0:04:23s
epoch 51 | loss: 57.4049 |  0:04:29s
epoch 52 | loss: 56.67611|  0:04:34s
epoch 53 | loss: 55.60401|  0:04:39s
e

epoch 44 | loss: 46.81725|  0:02:41s
epoch 45 | loss: 45.97834|  0:02:44s
epoch 46 | loss: 44.33474|  0:02:48s
epoch 47 | loss: 43.95131|  0:02:51s
epoch 48 | loss: 42.79397|  0:02:55s
epoch 49 | loss: 41.00698|  0:02:59s
epoch 50 | loss: 40.37275|  0:03:02s
epoch 51 | loss: 38.70666|  0:03:06s
epoch 52 | loss: 37.12836|  0:03:09s
epoch 53 | loss: 35.76505|  0:03:13s
epoch 54 | loss: 34.88873|  0:03:16s
epoch 55 | loss: 33.9664 |  0:03:20s
epoch 56 | loss: 33.149  |  0:03:23s
epoch 57 | loss: 32.80945|  0:03:26s
epoch 58 | loss: 31.21404|  0:03:30s
epoch 59 | loss: 31.06314|  0:03:33s
epoch 60 | loss: 30.3532 |  0:03:37s
epoch 61 | loss: 29.60607|  0:03:40s
epoch 62 | loss: 28.35645|  0:03:44s
epoch 63 | loss: 27.10131|  0:03:47s
epoch 64 | loss: 26.31028|  0:03:51s
epoch 65 | loss: 25.78284|  0:03:54s
epoch 66 | loss: 25.67565|  0:03:58s
epoch 67 | loss: 24.46742|  0:04:01s
epoch 68 | loss: 23.8152 |  0:04:05s
epoch 69 | loss: 23.0728 |  0:04:08s
epoch 70 | loss: 22.06534|  0:04:12s
e

In [ ]:
search_results = search_results.set_index(pd.Index([i for i in range(search_results.shape[0])]
min_mse_index= search_results['score'].idxmin()
best_model_row = search_results.iloc[min_mse_index,:]

best_model = best_model = TabNetRegressor(gamma = best_model_row["gamma"].values[0], 
               n_a = best_model_row["n_a"].values[0], 
               n_d = best_model_row["n_d"].values[0], 
               n_steps = best_model_row["n_steps"].values[0])
best_model.fit(X_train.to_numpy(), y_train.values.reshape(-1,1))

In [ ]:
score(y_test.to_numpy(), best_model.predict(X_test.to_numpy()))